# Imports and setting up viz

NB : conda env1 on PC, lam1env on spirit (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_mapping import *
from tools_routing import *

In [ ]:
rivers = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '10m',edgecolor=(0, 0, 0, 0.3), facecolor='none')

# Load and edit files

LAM output analysis.

simu irr et simu no-irr.

In [ ]:
date_min='2010-01-01'
date_max='2022-12-31'

In [ ]:
lon_min=-10
lon_max=4
lat_min=35.5
lat_max=44

## Sims


### LMDZ

In [ ]:
#flag to use time series of each used variable instead of full outputs file
TS_flag=False

In [ ]:
noirr_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/noirr'
irr_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/irr'

In [ ]:
#sim
if TS_flag:
    filename = '{}/*/ATM/TS_MO/*.nc'.format(noirr_dir)
else:
    filename = '{}/*/ATM/MO/*.nc'.format(noirr_dir)


sim0 = xr.open_mfdataset(filename)
sim0.attrs['name'] = 'no_irr'
sim = sim0.rename({'time_counter':'time'})
sim.attrs["plot_color"] = 'red'
sim = sim.sel(lon=slice(lon_min, lon_max),lat=slice(lat_min, lat_max))

sim['evap'] = sim['evap'] *3600 * 24
sim['evap'].attrs['units'] = 'mm d⁻¹'
sim['evap'].attrs['long_name'] = 'Evapotranspiration'

sim['precip'] = sim['precip'] *3600 * 24
sim['precip'].attrs['units'] = 'mm d⁻¹'
sim['precip'].attrs['long_name'] = 'Total precipitation'

sim['fluxsens']= -sim['sens']

sim['netrad'] = sim['LWdnSFC'] - sim['LWupSFC'] + sim['SWdnSFC'] - sim['SWupSFC']
sim['netrad'].attrs['units'] = 'W m⁻²'

sim['SWnetSFC'] = sim['SWdnSFC'] - sim['SWupSFC']
sim['SWnetSFC'].attrs['units'] = 'W m⁻²'

sim['LWnetSFC'] = sim['LWdnSFC'] - sim['LWupSFC']
sim['LWnetSFC'].attrs['units'] = 'W m⁻²'

sim['P - E'] = sim['precip'] - sim['evap']
sim['P - E'].attrs['units'] = 'mm d⁻¹'

sim['psol'] = sim['psol'] / 100
sim['psol'].attrs['units'] = 'hPa'

sim=compute_grid_cell_width(sim)
sim=add_moisture_divergence(sim)


if not TS_flag :
    sim['Altitude'] = sim['phis'] / 9.81
    sim['Altitude'].attrs['units'] = 'm'

sim

In [ ]:
#sim irr
if TS_flag:
    filename = '{}/*/ATM/TS_MO/*.nc'.format(irr_dir)
else:
    filename = '{}/*/ATM/MO/*.nc'.format(irr_dir)

sim0irr = xr.open_mfdataset(filename)
sim0irr.attrs['name'] = 'irr'
simirr = sim0irr.rename({'time_counter':'time'})
simirr.attrs["plot_color"] = 'blue'
simirr = simirr.sel(lon=slice(lon_min, lon_max),lat=slice(lat_min, lat_max))

simirr['evap'] = simirr['evap'] *3600 * 24
simirr['evap'].attrs['units'] = 'mm d⁻¹'

simirr['precip'] = simirr['precip'] *3600 * 24
simirr['precip'].attrs['units'] = 'mm d⁻¹'

simirr['fluxsens']= -simirr['sens']

simirr['netrad'] = simirr['LWdnSFC'] - simirr['LWupSFC'] + simirr['SWdnSFC'] - simirr['SWupSFC']
simirr['netrad'].attrs['units'] = 'W m⁻²'

simirr['SWnetSFC'] = simirr['SWdnSFC'] - simirr['SWupSFC']
simirr['SWnetSFC'].attrs['units'] = 'W m⁻²'

simirr['LWnetSFC'] = simirr['LWdnSFC'] - simirr['LWupSFC']
simirr['LWnetSFC'].attrs['units'] = 'W m⁻²'

simirr['P - E'] = simirr['precip'] - simirr['evap']
simirr['P - E'].attrs['units'] = 'mm d⁻¹'

simirr['psol'] = simirr['psol'] / 100
simirr['psol'].attrs['units'] = 'hPa'

simirr=compute_grid_cell_width(simirr)
simirr=add_moisture_divergence(simirr)

if not TS_flag:
    simirr['Altitude'] = simirr['phis'] / 9.81
    simirr['Altitude'].attrs['units'] = 'm'

simirr


### ORCHIDEE

In [ ]:
# #define evaporative fraction
# ip_ORCnoirr['evap_frac'] = ip_ORCnoirr['fluxlat'] / (ip_ORCnoirr['fluxlat'] + ip_ORCnoirr['fluxsens'])
# ip_ORCirr['evap_frac'] = ip_ORCirr['fluxlat'] / (ip_ORCirr['fluxlat'] + ip_ORCirr['fluxsens'])
# # add units
# ip_ORCnoirr['evap_frac'].attrs['units']=''
# ip_ORCirr['evap_frac'].attrs['units']=''

In [ ]:
def format_ORC_MO(filename, name, color, lon_min, lon_max, lat_min, lat_max):
    ds =  xr.open_mfdataset(filename)
    ds = ds.rename({'time_counter':'time'})
    ds.attrs['name'] = name
    ds.attrs['plot_color'] = color
    ds = ds.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

    ds['snowmelt'] = ds['snowmelt'] * 24 * 3600
    ds['snowmelt'].attrs['units'] = 'mm d⁻¹'

    ds['snow_contrib'] = ds['snowf'] / (ds['snowf'] + ds['rain']) * 100
    ds['snow_contrib'].attrs['units'] = '%'

    ds['precip'] = ds['rain'] + ds['snowf']
    ds['precip'].attrs['units'] = 'mm d⁻¹'

    ds['aei_sw'] = ds['aei_sw'] * 100

    ds['tot_runoff'] = ds['runoff'] + ds['drainage']
    ds['tot_runoff'].attrs['units'] = 'mm d⁻¹'
 
    ds['evap_frac'] = ds['fluxlat'] / (ds['fluxlat'] + ds['fluxsens'])
    ds['evap_frac'].attrs['units']='-'

    return ds

In [ ]:
if TS_flag:
    filename = '{}/*/SRF/TS_MO/*.nc'.format(noirr_dir)
elif not TS_flag:
    filename = '{}/*/SRF/MO/*sechiba_history.nc'.format(noirr_dir)

name='no_irr'
color='red'

ORCnoirr = format_ORC_MO(filename, name, color, lon_min, lon_max, lat_min, lat_max)

ORCnoirr

In [ ]:
if TS_flag:
    filename = '{}/*/SRF/TS_MO/*.nc'.format(noirr_dir)
elif not TS_flag:
    filename = '{}/*/SRF/MO/*history.nc'.format(irr_dir)

name = 'irr'
color = "#0C5DA5"

ORCirr = format_ORC_MO(filename, name, color, lon_min, lon_max, lat_min, lat_max)

ORCirr

In [ ]:
if not TS_flag:
    ORCnoirr['irrigmap_dyn'] = ORCirr['irrigmap_dyn']
    ORCnoirr['irrig_frac'] = ORCirr['irrigmap_dyn']/ORCirr['Areas'] * 100
    ORCnoirr['irrig_frac'].attrs['units'] = '%'
    ORCirr['irrig_frac'] = ORCirr['irrigmap_dyn']/ORCirr['Areas'] * 100
    ORCirr['irrig_frac'].attrs['units'] = '%'

### Routing

In [ ]:
rename_dict = {
        'time_counter' : 'time',
        'routing_stream_reservoir_r' : 'streamr',
        'routing_fast_reservoir_r' : 'fastr',
        'routing_slow_reservoir_r' : 'slowr',
        'routing_drainage_r' : 'drainage',
        'routing_runoff_r' : 'runoff',
        'routing_hydrographs_r' : 'hydrographs',
        'routing_irrigation_r' : 'irrigation',
        'irrig_netereq_r' : 'netirrig'
        }

long_name_dict = {
        'streamr' : 'stream reservoir',
        'fastr' : 'fast reservoir',
        'slowr' : 'slow reservoir',
        'drainage' : 'drainage',
        'runoff' : 'runoff',
        'hydrographs' : 'hydrographs',
        'irrigation' : 'irrigation',
        'netirrig' : 'net irrigation'
        }

In [ ]:
filename = '{}/*/SRF/MO/*diag_routing_r.nc'.format(noirr_dir)
routing_noirr = xr.open_mfdataset(filename)

routing_noirr = routing_noirr.rename(rename_dict)
routing_noirr = routing_noirr.sel(lon=slice(lon_min, lon_max),lat=slice(lat_max, lat_min))

#add long name to variables
for var in routing_noirr.variables:
    if var in long_name_dict.keys():
        routing_noirr[var].attrs['long_name'] = long_name_dict[var]
from dateutil.relativedelta import relativedelta

#move data 1 month back to correct for output file convention
routing_noirr['time'] = routing_noirr['time'].copy(
    data=[(pd.Timestamp(t.item()) - relativedelta(months=1)).to_datetime64() for t in routing_noirr['time'].values]
)

#add unit m³/s to hydrographs
routing_noirr['hydrographs'].attrs['units'] = 'm³/s'

routing_noirr.attrs['name'] = 'no_irr'
routing_noirr.attrs['plot_color'] = 'red'

routing_noirr

In [ ]:
filename = '{}/*/SRF/MO/*diag_routing_r.nc'.format(irr_dir)
routing_irr = xr.open_mfdataset(filename)

routing_irr = routing_irr.rename(rename_dict)
routing_irr = routing_irr.sel(lon=slice(lon_min, lon_max),lat=slice(lat_max, lat_min))

#add long name to variables
for var in routing_irr.variables:
    if var in long_name_dict.keys():
        routing_irr[var].attrs['long_name'] = long_name_dict[var]

#move data 1 month back to correct for output file convention
routing_irr['time'] = routing_irr['time'].copy(
    data=[(pd.Timestamp(t.item()) - relativedelta(months=1)).to_datetime64() for t in routing_irr['time'].values]
)

#add unit m³/s to hydrographs
routing_irr['hydrographs'].attrs['units'] = 'm³/s'

routing_irr.attrs['name']='irr'
routing_irr.attrs['plot_color'] = 'blue'

routing_irr

## Obs

### GLEAM

In [ ]:
filename = '../../../obs/GLEAMv4.1a/E/*'
gleam = xr.open_mfdataset(filename)
gleam.attrs['name'] = 'GLEAMv4.1a'
gleam.attrs['plot_color'] = 'black'
#restrict lon lat
gleam = gleam.sel(lon=slice(lon_min,lon_max),lat=slice(lat_max,lat_min))

gleam['evap']=convert_mm_per_month_to_mm_per_day(gleam['E'])
gleam['evap'].attrs['units'] = 'mm d⁻¹'
gleam

### GPCC

In [ ]:
#gpcc
filename='../../../obs/precips/precip.mon.total.0.25x0.25.v2020.nc'
gpcc0=xr.open_mfdataset(filename)
gpcc0.attrs['name'] = 'GPCC'
gpcc=gpcc0.sel(time=slice(date_min,date_max))
gpcc['lon'] = ((gpcc['lon'] + 180) % 360) - 180
gpcc1 = gpcc.sortby('lon')

gpcc=gpcc1.sel(lon=slice(-13,6.25),lat=slice(lat_max, lat_min))

gpcc.attrs["plot_color"] = 'black'

gpcc['precip'] = convert_mm_per_month_to_mm_per_day(gpcc['precip'])
gpcc['precip'].attrs['units'] = 'mm d⁻¹'
gpcc

### Ebro irrig

In [ ]:
load_Ebro_obs=True #false by default because it takes a lot of time to load

In [ ]:
if (load_Ebro_obs):
    filename = '../../../obs/SM_based_inversion_approach_EBRO_Irrigation.nc'
    obsEbro = xr.open_dataset(filename)
    obsEbro = obsEbro.rename({'Time':'time'})
    obsEbro = obsEbro.rename({'Longitude':'lon'})
    obsEbro = obsEbro.rename({'Latitude':'lat'})
    obsEbro = obsEbro.rename({'Irrigation':'irrigation'})

    obsEbro = regrid_to_lon_lat(obsEbro)
    obsEbro.attrs['name'] = 'Ebro_estimate'
    obsEbro.attrs['plot_color'] = "black"

    obsEbro

### Discharge

In [ ]:
filename = '../../../obs/streamflow/GRDC-Monthly_Spain.nc'
obs_routing = xr.open_dataset(filename)
obs_routing

### CCI

In [ ]:
filename='../../../obs/CCI-SM/cci_sm_monthly_filtered.nc'
cci = xr.open_mfdataset(filename)
cci['flag'].attrs['units']='-'

cci

## Masking, interpolation, and time period selection

### Sim masks

In [ ]:
#continental fraction mask (LMDZ outputs)
con_mask=sim.mean(dim='time')['contfracOR']>0.95
ip_mask=polygon_to_mask(sim, iberian_peninsula) * con_mask

In [ ]:
ip_sim = sim.where(ip_mask, drop=False)
ip_simirr = simirr.where(ip_mask, drop=False)

In [ ]:
ip_ORCirr=ORCirr.where(ip_mask, drop=False)
ip_ORCnoirr=ORCnoirr.where(ip_mask, drop=False)

### Interpolation and filtering of products

In [ ]:
if (load_Ebro_obs):
    ebro_mask = obsEbro['irrigation']>0
    ORCirr_iObsEbro = ORCirr.interp_like(obsEbro)
    ebro_ORCirr_iObsEbro = ORCirr_iObsEbro.where(ebro_mask)
    obsEbro_iORC = obsEbro.interp_like(ORCirr)
    ebro_mask_iORCirr = obsEbro_iORC['irrigation']>0
    ebro_ORCirr = ORCirr.where(ebro_mask_iORCirr)

In [ ]:
#gleam
gleam_isim=gleam.interp_like(sim)
cont_gleam_isim=gleam_isim.where(con_mask)
ip_gleam_isim = cont_gleam_isim.where(ip_mask, drop=False)

In [ ]:
#gpcc
gpcc_isim=gpcc.interp_like(sim)
cont_gpcc_isim=gpcc_isim.where(con_mask)
ip_gpcc_isim = cont_gpcc_isim.where(ip_mask, drop=False)

In [ ]:
## CCI
cci_iORC = cci.interp_like(ORCnoirr)
ip_cci_iORC = cci_iORC.where(ip_mask)

# Figures

## Fig 1 : Domain and altitude

In [ ]:
import psyplot.project as psy
psy.rcParams['auto_show'] = True
mpl.rcParams['figure.figsize'] = [12, 9.]

In [ ]:
LAM_dir='../../../JZ_simu_outputs/LAM/native_outputs/'
start_file='{}/espagne_start_2010.nc'.format(LAM_dir)
start=xr.open_dataset(start_file)
# start

In [ ]:
start["Altitude"] = start["phis"]/9.81
start["Altitude"].attrs["units"] = "m"
start["Altitude"].attrs["long_name"] = "Altitude"
start["Altitude"].attrs["standard_name"] = "Altitude"

start_file2='{}/espagne_start_2010_modified.nc'.format(LAM_dir)
start.to_netcdf(start_file2)

In [ ]:
start2=xr.open_dataset(start_file2)
# start2

In [ ]:
# Custom colormap
terrain_colors = plt.cm.terrain(np.linspace(0, 1, 25))
filtered_colors = terrain_colors[5:]  
# Insert blue as the first color
filtered_colors = np.vstack([[0.07973856209150328, 0.44052287581699345, 0.8405228758169935, 1.0], filtered_colors]) 
custom_colormap = ListedColormap(filtered_colors)


In [ ]:
file=start_file2
var='Altitude'
vmin=-0
vmax=2000
pas=(vmax-vmin) / 20

map=psy.plot.mapplot(
    file, 
    name=var,
    datagrid=dict(color='k', linewidth=0.2),
    cbar='r',
    tight=True,
    lsm='50m',
    cmap=custom_colormap,
    extend='both',
    projection='ortho',
    xgrid=True,ygrid=True,
    bounds=np.arange(vmin, vmax + pas, pas),
    # title=var,
    clabel="Altitude (m)"
)

## Fig 2 : Stations and dams
NB : This figure was finally edited differently with the help of GIS engineer Aurélien Baro (METIS).

In [ ]:
proper_stations_dict = {
    6226800: {'name': 'Tortosa',            'river': 'Ebro',            'lat_grid': 40.82500,   'lon_grid': 0.525007,   'station_nb': 1},
    6226400: {'name': 'Zaragoza',           'river': 'Ebro',            'lat_grid': 41.67499,   'lon_grid': -0.90832,   'station_nb': 2},
    6226300: {'name': 'Castejon',           'river': 'Ebro',            'lat_grid': 42.17499,   'lon_grid': -1.69165,   'station_nb': 3},
    6226600: {'name': 'Seros',              'river': 'Segre',           'lat_grid': 41.45833,   'lon_grid': 0.425007,   'station_nb': 4},
    6226650: {'name': 'Fraga',              'river': 'Cinca',           'lat_grid': 41.52499,   'lon_grid': 0.341674,   'station_nb': 5},
    6212410: {'name': 'Tore',               'river': 'Douro',           'lat_grid': 41.50833,   'lon_grid': -5.47499,   'station_nb': 6},
    6212700: {'name': 'Peral De Arlanza',   'river': 'Arlanza',         'lat_grid': 42.07500,   'lon_grid': -4.07499,   'station_nb': 7},
    6213700: {'name': 'Talavera',           'river': 'Tagus',           'lat_grid': 39.95833,   'lon_grid': -4.82499,   'station_nb': 8},
    6213800: {'name': 'Trillo',             'river': 'Tagus',           'lat_grid': 40.70833,   'lon_grid': -2.57499,   'station_nb': 9},
    6213900: {'name': 'Peralejos',          'river': 'Tagus',           'lat_grid': 40.59166,   'lon_grid': -1.92499,   'station_nb': 10},
    6216510: {'name': 'Azud de Badajoz',    'river': 'Guadiana',        'lat_grid': 38.86199,   'lon_grid': -7.01,      'station_nb': 11}, 
    6116200: {'name': 'Pulo do Lobo',       'river': 'Guadiana',        'lat_grid': 37.803,     'lon_grid': -7.633,     'station_nb': 12},         
    6216530: {'name': 'La Cubeta',          'river': 'Guadiana',        'lat_grid': 38.975,     'lon_grid': -2.895,     'station_nb': 13},         
    6216520: {'name': 'Villarubia',         'river': 'Guadiana',        'lat_grid': 39.125,     'lon_grid': -3.59073,   'station_nb': 14},      
    6216800: {'name': 'Quintanar',          'river': 'Giguela',         'lat_grid': 39.64166,   'lon_grid': -3.07499,   'station_nb': 15},
    6217140: {'name': 'Mengibar',           'river': 'Guadalquivir',    'lat_grid': 37.98425,   'lon_grid': -3.79939,   'station_nb': 16},     
    6217200: {'name': 'Arroyo Maria',       'river': 'Guadalquivir',    'lat_grid': 38.17905,   'lon_grid': -2.83594,   'station_nb': 17}, 
    6217700: {'name': 'Pinos Puente',       'river': 'Frailes',         'lat_grid': 37.27499,   'lon_grid': -3.75832,   'station_nb': 18},
}

#keeping only 5 representative stations for larger rivers, for simple figure
representative_stations_dict = {
    6226800: {'name': 'Tortosa',            'river': 'Ebro',            'lat_grid': 40.82500,   'lon_grid': 0.525007,   'station_nb': 1},
    6212410: {'name': 'Tore',               'river': 'Douro',           'lat_grid': 41.50833,   'lon_grid': -5.47499,   'station_nb': 6},
    6213700: {'name': 'Talavera',           'river': 'Tagus',           'lat_grid': 39.95833,   'lon_grid': -4.82499,   'station_nb': 8},
    6216510: {'name': 'Azud de Badajoz',    'river': 'Guadiana',        'lat_grid': 38.86199,   'lon_grid': -7.01,      'station_nb': 11}, 
    6217140: {'name': 'Mengibar',           'river': 'Guadalquivir',    'lat_grid': 37.98425,   'lon_grid': -3.79939,   'station_nb': 16},     
}

In [ ]:
filename='../../../obs/Europe-dams_edited.ods'
dams = pd.read_excel(filename, engine='odf')
mask = dams['Country']=='Spain'
dams = dams[mask]
rename_dict = {
    'Name of dam': 'Name',
    'Decimal degree latitude': 'lat',
    'Decimal degree longitude': 'lon',
    'Reservoir capacity (million m3)' : 'capacity'
}
dams.rename(columns=rename_dict, inplace=True)

# dams

In [ ]:
# --- Configuration ---
# Set the path to your main Shapefile (.shp)
RBD_FILE_PATH = '../../../obs/RiverBasinDistrict/RiverBasinDistrict.shp'
# ISO 3166-1 alpha-2 code for Spain
COUNTRY_CODE = ['ES','PT']
# COUNTRY_CODE = ['ES']
# The column in the shapefile's attribute table that contains the country code
# This is a common name in European datasets; you might need to check your .dbf if 'country' fails.
COUNTRY_CODE_COLUMN = 'country'
# --- 1. Data Loading and Filtering ---
print("Loading River Basin Districts data...")
try:
    # 1. Load the Shapefile using GeoPandas
    # GeoPandas reads all associated files (.dbf, .shx) automatically.
    rbd_europe = gpd.read_file(RBD_FILE_PATH)
    
    # Ensure the GeoDataFrame is in the standard geographic CRS (WGS84)
    rbd_europe = rbd_europe.to_crs(ccrs.PlateCarree())
    
    # 2. Filter the data to keep only Spanish territories
    print(f"Filtering data for Spain ({COUNTRY_CODE})...")
    # We use .query() to select rows where the country code column matches 'ES'
    # Use .str.upper() to handle potential case inconsistencies in the attribute table
    rbd_spain = rbd_europe[rbd_europe[COUNTRY_CODE_COLUMN].str.upper().isin(COUNTRY_CODE)]
    
    if rbd_spain.empty:
        print(f"\n! Warning: No catchments found for code '{COUNTRY_CODE}' in column '{COUNTRY_CODE_COLUMN}'.")
        print("Please check the attribute table (the .dbf file) of your Shapefile to confirm the correct country code column name (e.g., 'country', 'CNTR', 'COUNTRY', etc.).")
        # Fallback for demonstration if filtering fails (plots entire continent's extent)
        # In a real scenario, you'd stop here until the column is fixed.
        rbd_spain = rbd_europe[rbd_europe['geometry'].bounds.minx < 5] # Crude filter for European West
        
except FileNotFoundError:
    print(f"\n! ERROR: Shapefile not found at '{RBD_FILE_PATH}'.")
    print("Please ensure 'RiverBasinDistrict.shp' and its accompanying files are in the same directory as this script.")
    exit()
except KeyError as e:
    print(f"\n! ERROR: Column {e} not found in the shapefile attributes.")
    print("You need to inspect the attribute table of your shapefile to find the correct column name for the country code and update the `COUNTRY_CODE_COLUMN` variable.")
    exit()
except Exception as e:
    print(f"\nAn unexpected error occurred during data loading: {e}")
    exit()

# rbd_spain

In [ ]:
stations_map_dict(proper_stations_dict, river_cond=None, legend=True,
                            dams_df=dams, dam_nb=252, title=None,
                            extent=[-10, 2.5, 36, 44],
                            catchment_df=rbd_spain
                            )
plt.savefig('figures/article/f02.png')

## Fig 3 : Irrigation maps

In [ ]:
vars=['irrig_frac', 'aei_sw', 'netirrig', 'irrigation', 'slowr', 'streamr']
cmaps=[reds, emb_neutral, blues, wet, emb_neutral, emb_neutral]
vmins=[0, 0, 0, 0, -100, -100]
vmaxs=[50, 100, 10, 0.6, 100, 100]
clabels=['Grid cell irrigated fraction (%)',
         'Share of surface equipments (%)',
         'Irrigation requirement (mm d⁻¹)',
         'Irrigation (mm d⁻¹)',
         'Groundwater reservoir change (%)',
         'River reservoir change (%)']

fig, axes = plt.subplots(3, 2, figsize=(17,12), subplot_kw={'projection': ccrs.PlateCarree()})
axes = axes.flatten()
for i in range(4):
    var=vars[i]
    plotvar=ip_ORCirr[var].mean(dim='time')
    ax=axes[i]
    cmap=cmaps[i]
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel=clabels[i]
    nice_map(plotvar, ax, cmap, vmin, vmax, clabel=clabel)
    title='({})'.format(letters[i])
    ax.set_title(title)

for i in range(4, 6):
    var=vars[i]
    rel_diff = (ip_ORCirr[var] - ip_ORCnoirr[var]).mean(dim='time') / ip_ORCnoirr[var].mean(dim='time') * 100
    plotvar= rel_diff
    ax=axes[i]
    cmap=cmaps[i]
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel=clabels[i]
    nice_map(plotvar, ax, cmap, vmin, vmax, clabel=clabel)
    title='({})'.format(letters[i])
    ax.set_title(title)

## Fig 4 : Irrig eval vs Dari et al

In [ ]:
obsEbro['irrigation'] = convert_mm_per_month_to_mm_per_day(obsEbro['irrigation'])

In [ ]:
if load_Ebro_obs:
    var='irrigation'
    ds1=obsEbro.sel(time=slice('2016-01-01','2020-07-31'))
    ds2=ebro_ORCirr.sel(time=slice('2016-01-01','2020-07-31'))
    ds3=ebro_ORCirr_iObsEbro.sel(time=slice('2016-01-01','2020-07-31'))

In [ ]:
if load_Ebro_obs:
    fig = plt.figure(figsize=(8.5, 9))
    gs = gridspec.GridSpec(2, 1, height_ratios=[1, 1])

    # Seasonal cycle
    ax1 = fig.add_subplot(gs[0, 0]) 
    ylabel = 'Irrigation (mm d⁻¹)'

    plotvar1 = ds1[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
    color1 = 'black'
    mean1 = plotvar1.mean().values
    label1 = f'Ebro_estimate ({mean1:.2f} mm d⁻¹)'
    nice_time_plot(plotvar1, ax1, label=label1, color=color1, ylabel=ylabel)

    plotvar2 = ds2[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
    color2 = 'blue'
    mean2 = plotvar2.mean().values
    label2 = f'irr ({mean2:.2f} mm d⁻¹)'
    nice_time_plot(plotvar2, ax1, label=label2, color=color2, ylabel=ylabel)

    ax1.set_xticks(np.arange(1, 13))
    ax1.set_xticklabels(months_name_list)
    ax1.set_title('(a)')

    # Map
    ax2 = fig.add_subplot(gs[1, 0], projection=ccrs.PlateCarree())
    plotvar = (ds3[var] - ds1[var]).mean(dim='time')
    cmap = emb_neutral
    vmin = -0.5
    vmax = 0.5
    clabel = 'Irrigation bias (mm d⁻¹)'

    nice_map(plotvar, ax2, cmap, vmin, vmax, clabel=clabel)
    ax2.set_title('(b)')

    plt.tight_layout()

## Fig 5 : 5 stations

In [ ]:
#plot a seasonnal cycle for each station from representative_stations_dict
fig, axes= plt.subplots(5,1, figsize=(10,25))
axes= axes.flatten()
ds_list=[routing_noirr, routing_irr]
for i, (station_id, station_data) in enumerate(representative_stations_dict.items()):
    title_letter=letters[i]
    sc_station(obs_routing,axes[i], station_id, name=station_data['name'], ylabel='River discharge (m³ s⁻¹)', year_min=2010, year_max=2022, show_mean=True)
    sc_with_obs(ds_list, obs_routing, axes[i], station_id, station_data, ylabel='River discharge (m³ s⁻¹)', year_min=2010, year_max=2022, title_letter=title_letter, show_mean=True)

## Fig 6 : ET and P eval vs GPCC and GLEAM

In [ ]:
## REVISED VERSION (including SSM)
fig = plt.figure(figsize=(18, 12))
gs = gridspec.GridSpec(3, 3, width_ratios=[1.1,1,1.2], height_ratios=[1, 1, 1])

vmin_p=-1
vmax_p= 1
vmin_et=-1
vmax_et= 1
vmin_sm = -1.5
vmax_sm=1.5


## PRECIP ##
date_min = '2010-01-01'
date_max = '2019-12-31'
ds_obs=ip_gpcc_isim.sel(time=slice(date_min,date_max))
ds_irr=ip_simirr.sel(time=slice(date_min,date_max))
ds_noirr=ip_sim.sel(time=slice(date_min,date_max))
var='precip'
# Seasonal cycle
ax1 = fig.add_subplot(gs[0,0])
ylabel="Precipitation (mm d⁻¹)"
plotvar1 = ds_obs[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
color1 = 'black'
mean1 = plotvar1.mean().values
label1 = f'GPCC ({mean1:.2f} mm d⁻¹)'
nice_time_plot(plotvar1, ax1, vmin=0, label=label1, color=color1, ylabel=ylabel)
plotvar2 = ds_noirr[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
color2 = 'red'
mean2 = plotvar2.mean().values
label2 = f'no_irr ({mean2:.2f} mm d⁻¹)'
nice_time_plot(plotvar2, ax1, vmin=0, label=label2, color=color2, ylabel=ylabel)
plotvar3 = ds_irr[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
color3 = 'blue'
mean3 = plotvar3.mean().values
label3 = f'irr ({mean3:.2f} mm d⁻¹)'
nice_time_plot(plotvar3, ax1, vmin=0, vmax=3.5, label=label3, color=color3, ylabel=ylabel)
ax1.set_title('(a) Mean seasonnal cycle (2010-2019)')
ax1.set_xticks(np.arange(1, 13))
ax1.set_xticklabels(months_name_list)

#Diff
ax2 = fig.add_subplot(gs[0,1], projection=ccrs.PlateCarree())
plotvar=(ds_noirr[var]-ds_obs[var]).mean(dim='time')
cmap=emb_neutral
vmin=vmin_p
vmax=vmax_p
clabel="Precipitation bias (mm d⁻¹)"
nice_map(plotvar, ax2, cmap, vmin, vmax, clabel=clabel, cbar_on=False)
ax2.set_title('(b) no_irr - GPCC')

ax3 = fig.add_subplot(gs[0,2], projection=ccrs.PlateCarree())
plotvar=(ds_irr[var]-ds_obs[var]).mean(dim='time')
cmap=emb_neutral
vmin=vmin_p
vmax=vmax_p
clabel="Precipitation bias (mm d⁻¹)"
nice_map(plotvar, ax3, cmap, vmin, vmax, clabel=clabel, left_labels=False)
ax3.set_title('(c) irr - GPCC')


## ET ##
date_min = '2010-01-01'
# date_max = '2022-12-31'
date_max = '2019-12-31' # to match with GPCC
ds_obs=ip_gleam_isim.sel(time=slice(date_min,date_max))
ds_irr=ip_simirr.sel(time=slice(date_min,date_max))
ds_noirr=ip_sim.sel(time=slice(date_min,date_max))
var='evap'
# Seasonal cycle
ax4 = fig.add_subplot(gs[1,0])
vmin=0
ylabel="Evapotranspiration (mm d⁻¹)"
plotvar1 = ds_obs[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
mean1 = plotvar1.mean().values
label1 = f'GLEAM ({mean1:.2f} mm d⁻¹)'
color1 = 'black'
nice_time_plot(plotvar1, ax4, vmin=0, label=label1, color=color1, ylabel=ylabel)
plotvar2 = ds_noirr[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
color2 = 'red'
mean2 = plotvar2.mean().values
label2 = f'no_irr ({mean2:.2f} mm d⁻¹)'
nice_time_plot(plotvar2, ax4, vmin=0, label=label2, color=color2, ylabel=ylabel)
plotvar3 = ds_irr[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
color3 = 'blue'
mean3 = plotvar3.mean().values
label3 = f'irr ({mean3:.2f} mm d⁻¹)'
nice_time_plot(plotvar3, ax4, vmin=0, vmax=3.5, label=label3, color=color3, ylabel=ylabel)

ax4.set_title('(d) Mean seasonnal cycle (2010-2019)')
ax4.set_xticks(np.arange(1, 13))
ax4.set_xticklabels(months_name_list)

#Diff
ax5 = fig.add_subplot(gs[1,1], projection=ccrs.PlateCarree())
plotvar=(ds_noirr[var]-ds_obs[var]).mean(dim='time')
cmap=emb_neutral
vmin=vmin_et
vmax=vmax_et
clabel="Evapotranspiration bias (mm d⁻¹)"
nice_map(plotvar, ax5, cmap, vmin, vmax, clabel=clabel, cbar_on=False)
ax5.set_title('(e) no_irr - GLEAM')

ax6 = fig.add_subplot(gs[1,2], projection=ccrs.PlateCarree())
plotvar=(ds_irr[var]-ds_obs[var]).mean(dim='time')
cmap=emb_neutral
vmin=vmin_et
vmax=vmax_et
clabel="Evapotranspiration bias (mm d⁻¹)"
nice_map(plotvar, ax6, cmap, vmin, vmax, clabel=clabel, left_labels=False)
ax6.set_title('(f) irr - GLEAM')


## NORMALIZED SSM ##
date_min = '2010-01-01'
# date_max = '2022-12-31'
date_max = '2019-12-31' # to match with GPCC
ds_obs=ip_cci_iORC.sel(time=slice(date_min,date_max))
ds_irr=ip_ORCirr.sel(time=slice(date_min,date_max))
ds_noirr=ip_ORCnoirr.sel(time=slice(date_min,date_max))
var='norm_sm'

#compute normalized SSM over the given period
mrsos_mean_noirr    = ds_noirr['mrsos'].mean(dim=['time','lon','lat']).values
mrsos_std_noirr     = ds_noirr['mrsos'].std(dim=['time','lon','lat']).values
ds_noirr['norm_sm'] = (ds_noirr['mrsos'] - mrsos_mean_noirr) / mrsos_std_noirr
ds_noirr['norm_sm'].attrs['units'] = '-'


mrsos_mean_irr    = ds_irr['mrsos'].mean(dim=['time','lon','lat']).values
mrsos_std_irr     = ds_irr['mrsos'].std(dim=['time','lon','lat']).values
ds_irr['norm_sm'] = (ds_irr['mrsos'] - mrsos_mean_irr) / mrsos_std_irr
ds_irr['norm_sm'].attrs['units'] = '-'

ssm_mean    = ds_obs['sm'].mean(dim=['time','lon','lat']).values
ssm_std     = ds_obs['sm'].std(dim=['time','lon','lat']).values
ds_obs['norm_sm'] = (ds_obs['sm'] - ssm_mean) / ssm_std
ds_obs['norm_sm'].attrs['units'] = '-'

# Seasonal cycle
ax7 = fig.add_subplot(gs[2,0])
vmin_sm_ts=-1.3
vmax_sm_ts=1.3
ylabel="Normalized SSM (-)"
plotvar1 = ds_obs[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
color1 = 'black'
label1 = 'CCI'
nice_time_plot(plotvar1, ax7, vmin=vmin_sm_ts, label=label1, color=color1, ylabel=ylabel)
plotvar2 = ds_noirr[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
color2 = 'red'
label2 = 'no_irr'
nice_time_plot(plotvar2, ax7, vmin=vmin_sm_ts, label=label2, color=color2, ylabel=ylabel)
plotvar3 = ds_irr[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
color3 = 'blue'
label3 = 'irr'
nice_time_plot(plotvar3, ax7, vmin=vmin_sm_ts, vmax=vmax_sm_ts, label=label3, color=color3, ylabel=ylabel)
ax7.set_title('(g) Mean seasonnal cycle (2010-2019)')
ax7.set_xticks(np.arange(1, 13))
ax7.set_xticklabels(months_name_list)

#Diff
ax8 = fig.add_subplot(gs[2,1], projection=ccrs.PlateCarree())
plotvar=(ds_noirr[var]-ds_obs[var]).mean(dim='time')
cmap=emb_neutral
vmin=vmin_sm
vmax=vmax_sm
clabel="Normalized SSM bias (-)"
nice_map(plotvar, ax8, cmap, vmin, vmax, clabel=clabel, cbar_on=False)
ax8.set_title('(h) no_irr - CCI')

ax9 = fig.add_subplot(gs[2,2], projection=ccrs.PlateCarree())
plotvar=(ds_irr[var]-ds_obs[var]).mean(dim='time')
cmap=emb_neutral
vmin=vmin_sm
vmax=vmax_sm
clabel="Normalized SSM bias (-)"
nice_map(plotvar, ax9, cmap, vmin, vmax, clabel=clabel, left_labels=False)
ax9.set_title('(i) irr - CCI')

plt.tight_layout()

## Fig 7 : Diffs irr - no_irr

In [ ]:
#With JJA irrig and wind
vars=['irrigation', 'evap_frac', 't2m', 's_pblh', 's_lcl', 'precip', 'cape', 'moisture_convergence']
vmins=[0  , -0.3, -0.35, -100, -250, -1, -35, -1]
vmaxs=[1.5,  0.3,  0.35,  100,  250,  1,  35,  1]
clabels=['Irrigation (mm d⁻¹)',
         'Evaporative fraction change',
         '2-m temperature change (K)',
         'ABL height change (m)',
         'LCL change (m)',
         'Precipitation change (mm d⁻¹)',
         'CAPE change (J kg⁻¹)', 
         'Convergence change (mm d⁻¹)']
cmaps=[wet, emb_neutral, emb, emb, emb, emb_neutral, emb, emb_neutral]

fig, axes = plt.subplots(4, 2, figsize=(17,16), subplot_kw={'projection': ccrs.PlateCarree()})
axes = axes.flatten()

months=[6, 7, 8]

i=0
map_wind(ip_simirr.sel(time=ip_simirr['time.month'].isin(months)), 
         axes[i], 
         extra_var=vars[i], 
         extra_ds=ip_ORCirr.sel(time=ip_ORCirr['time.month'].isin(months)).mean(dim='time'),
         height='10m', dist=2, scale=30,
         cmap=cmaps[i], vmin=vmins[i],vmax=vmaxs[i], clabel=clabels[i],
         quiver_inside=True)

pvalue=0.05
sig_method=1
check_norm=False
sig_viz=6
hatch='///'

for i in range(1,2):
    ds1=ip_ORCirr.sel(time=ip_ORCirr['time.month'].isin(months))
    ds2=ip_ORCnoirr.sel(time=ip_ORCnoirr['time.month'].isin(months))
    var=vars[i]
    diff = (ds1[var] - ds2[var]).mean(dim='time')
    plotvar= diff
    ax=axes[i]
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel=clabels[i]
    cmap=cmaps[i]
    sig_mask = compute_sig_mask(ds1, ds2, var, check_norm=check_norm, method=sig_method, pvalue=pvalue)
    nice_map(plotvar, ax, cmap=cmap, vmin=vmin, vmax=vmax, sig_mask=sig_mask, clabel=clabel, hatch=hatch)


for i in range(2, 8):
    ds1=ip_simirr.sel(time=ip_simirr['time.month'].isin(months))
    ds2=ip_sim.sel(time=ip_sim['time.month'].isin(months))
    var=vars[i]
    diff = (ds1[var] - ds2[var]).mean(dim='time')
    plotvar= diff
    ax=axes[i]
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel=clabels[i]
    cmap=cmaps[i]
    sig_mask = compute_sig_mask(ds1, ds2, var, check_norm=check_norm, method=sig_method, pvalue=pvalue)
    nice_map(plotvar, ax, cmap=cmap, vmin=vmin, vmax=vmax, sig_mask=sig_mask, clabel=clabel, hatch=hatch)


for i in range(8):
    title='({})'.format(letters[i])
    axes[i].set_title(title)


if True:
    plt.savefig('figures/article/f07.png', bbox_inches='tight', dpi=300)

## Fig 8 : Scatter plot ET and P changes

In [ ]:
ds1=ip_ORCirr.mean(dim=['lon', 'lat'])
ds2=(ip_ORCirr - ip_ORCnoirr).mean(dim=['lon', 'lat'])

In [ ]:
fig, axes= plt.subplots(1,2, figsize=(18,6))
axes=axes.flatten()
var1='irrigation'
xlabel='Irrigation (mm d⁻¹)'
seasons=['DJF', 'MAM', 'JJA', 'SON']

#evap vs irrig
ax=axes[0]
var2='evap'
ylabel='Evapotranspiration change (mm d⁻¹)'
title='(a)'
scatter_vars_seasons_ax(ax,ds1, ds2, var1, var2, reg=True, plot_one=True, title=title, coloring=True,
                      is_1D=True, seasons_list=seasons, xlabel=xlabel, ylabel=ylabel)
#precip vs irrig
ax=axes[1]
var2='rain'
ylabel='Precipitation change (mm d⁻¹)'
title='(b)'
scatter_vars_seasons_ax(ax,ds1, ds2, var1, var2, reg=True, plot_one=True, title=title, coloring=True,
                      is_1D=True, seasons_list=seasons, xlabel=xlabel, ylabel=ylabel)

## Fig 9 : Moisture budget by zone

### Define datasets and masks

In [ ]:
# Define 2D masks
# Applied irrigation
irr_mask_low    = ip_ORCirr.mean(dim='time')['irrigation']<0.06
irr_mask_high   = ip_ORCirr.mean(dim='time')['irrigation']>0.12
irr_mask_med    = (~irr_mask_low) * (~irr_mask_high) * ip_mask

In [ ]:
# Define datasets orc
lowirr_ip_ORCnoirr = apply_2Dmask_to_dataset(ip_ORCnoirr, irr_mask_low, dsname='lowirr_noirr')
lowirr_ip_ORCirr = apply_2Dmask_to_dataset(ip_ORCirr, irr_mask_low, dsname='lowirr_irr')

medirr_ip_ORCnoirr = apply_2Dmask_to_dataset(ip_ORCnoirr, irr_mask_med, dsname='medirr_noirr')
medirr_ip_ORCirr = apply_2Dmask_to_dataset(ip_ORCirr, irr_mask_med, dsname='medirr_irr')

highirr_ip_ORCnoirr = apply_2Dmask_to_dataset(ip_ORCnoirr, irr_mask_high, dsname='highirr_noirr')
highirr_ip_ORCirr = apply_2Dmask_to_dataset(ip_ORCirr, irr_mask_high, dsname='highirr_irr')

In [ ]:
# Temporal and spatial mean 
# for IP
ip_ORCnoirr_mean = ip_ORCnoirr.mean(dim=['time','lon','lat'])
ip_ORCirr_mean = ip_ORCirr.mean(dim=['time','lon','lat'])

#by irrig class
lowirr_ip_ORCnoirr_mean     = lowirr_ip_ORCnoirr.mean(dim=['time','lon','lat'])
lowirr_ip_ORCirr_mean       = lowirr_ip_ORCirr.mean(dim=['time','lon','lat'])
medirr_ip_ORCnoirr_mean     = medirr_ip_ORCnoirr.mean(dim=['time','lon','lat'])
medirr_ip_ORCirr_mean       = medirr_ip_ORCirr.mean(dim=['time','lon','lat'])
highirr_ip_ORCnoirr_mean    = highirr_ip_ORCnoirr.mean(dim=['time','lon','lat'])
highirr_ip_ORCirr_mean      = highirr_ip_ORCirr.mean(dim=['time','lon','lat'])

In [ ]:
#diff
#over whole IP
ip_orc_diff_mean = ip_ORCirr_mean - ip_ORCnoirr_mean

#by irrig class
lowirr_ip_orc_diff_mean     = lowirr_ip_ORCirr_mean - lowirr_ip_ORCnoirr_mean
medirr_ip_orc_diff_mean     = medirr_ip_ORCirr_mean - medirr_ip_ORCnoirr_mean
highirr_ip_orc_diff_mean    = highirr_ip_ORCirr_mean -  highirr_ip_ORCnoirr_mean

### Compute and display data

In [ ]:
compute_dict=True

In [ ]:
if compute_dict:
    data_dict = {
        "Variable": ["Irrigation",  "ET change","P change"],
        "Low irrigation":               [0,0,0],
        "Medium irrigation":            [0,0,0],
        "High irrigation":              [0,0,0],
        "Iberian Peninsula":            [0,0,0]
    }
    # add average values for irrigation, ET chang and P change to dictionary
    data_dict['Low irrigation'][0] = lowirr_ip_ORCirr_mean['irrigation'].compute().item()
    data_dict['Low irrigation'][1] = lowirr_ip_orc_diff_mean['evap'].compute().item()
    data_dict['Low irrigation'][2] = lowirr_ip_orc_diff_mean['rain'].compute().item()

    data_dict['Medium irrigation'][0] = medirr_ip_ORCirr_mean['irrigation'].compute().item()
    data_dict['Medium irrigation'][1] = medirr_ip_orc_diff_mean['evap'].compute().item()
    data_dict['Medium irrigation'][2] = medirr_ip_orc_diff_mean['rain'].compute().item()

    data_dict['High irrigation'][0] = highirr_ip_ORCirr_mean['irrigation'].compute().item()
    data_dict['High irrigation'][1] = highirr_ip_orc_diff_mean['evap'].compute().item()
    data_dict['High irrigation'][2] = highirr_ip_orc_diff_mean['rain'].compute().item()

    data_dict['Iberian Peninsula'][0] = ip_ORCirr_mean['irrigation'].compute().item()
    data_dict['Iberian Peninsula'][1] = ip_orc_diff_mean['evap'].compute().item()
    data_dict['Iberian Peninsula'][2] = ip_orc_diff_mean['rain'].compute().item()

    print(data_dict)
    
else:
    #variables already computed and saved for each subdataset (mm/d)
    data_dict = {'Variable': ['Irrigation', 'ET change', 'P change'],
                 'Low irrigation': [0.03250407055020332, 0.04089963436126709, 0.022051572799682617],
                 'Medium irrigation': [0.08199943602085114, 0.09069693088531494, 0.010108709335327148],
                 'High irrigation': [0.21001876890659332, 0.2211529016494751, 0.009575486183166504],
                 'Iberian Peninsula': [0.06673634797334671, 0.07550418376922607, 0.0167844295501709]
                 }


In [ ]:
#turn dict to df
irrZone_df = pd.DataFrame(data_dict)
irrZone_df

In [ ]:
#edit titles
irrZone_df['(a) Low irrigation'] = irrZone_df['Low irrigation']
irrZone_df['(b) Medium irrigation'] = irrZone_df['Medium irrigation']
irrZone_df['(c) High irrigation'] = irrZone_df['High irrigation']
irrZone_df['(d) Iberian Peninsula'] = irrZone_df['Iberian Peninsula']
irrZone_df.drop(columns=irrZone_df.columns[1:5], inplace=True) #drop 4 initial columns
irrZone_df


In [ ]:
make_combined_figure9(
    irrZone_df,
    ip_ORCirr,
    irr_mask_low,
    irr_mask_med,
    irr_mask_high,
    domain_labels=('Low irrigation', 'Medium irrigation', 'High irrigation')
)

# Tables

## Table 1 : Gridded products
Not made using Python

## Tab 2 : Discharge stations

In [ ]:
#make csv from proper_stations_dict
filename='../../../obs/stations_data.csv'
df = pd.DataFrame(proper_stations_dict).T
df.to_csv(filename)

## Tab 3 : Change in evaluated metrics

In [ ]:
#compute all metrics on one station (necessary for later automatic use of metrics)
station_id, station = next(iter(proper_stations_dict.items()))
metric_list=[metric_sim_module, metric_obs_module, metric_bias, metric_rmse, metric_tcorr, metric_nse, metric_kge]
for metric_to_use in metric_list:
    metric_value=compute_metric_station(routing_noirr, obs_routing, station_id, station, metric_to_use)
    print('{} for station {} : {}'.format(metric_to_use.__short_name__, station['name'], metric_value))

In [ ]:
# output a pandas dataframe with all metric values for a given list of metrics and stations
sim=routing_noirr
metric_list=[metric_sim_module, metric_obs_module, metric_bias, metric_rmse, metric_tcorr, metric_nse, metric_kge]
stations_dict=proper_stations_dict
# define dataframe with one row per station and one column per metric
df=[]
for station_id, station in stations_dict.items():
    label='{} ({})'.format(station['station_nb'],station['name'])
    df.append({'Station':label})
    for metric in metric_list:
        name=metric.__short_name__
        metric_value=compute_metric_station(sim, obs_routing, station_id, station, metric)
        metric_value=np.round(metric_value, 2)
        # append metric_value to df
        df[-1][name]=metric_value

# convert df to pandas dataframe
df_noirr=pd.DataFrame(df)
# set Station as index
df_noirr.set_index('Station', inplace=True)
df_noirr['Bias (%)'] = np.round(df_noirr['Bias (m³/s)'] / df_noirr['Module (obs, m³/s)'], 3) * 100
#move Bias (%) to the 4th column
cols = df_noirr.columns.tolist()
cols = cols[:3] + cols[-1:] + cols[3:-1]
df_noirr = df_noirr[cols]
#add average row
df_noirr.loc['Mean'] = df_noirr.mean()
df_noirr.drop('Module (sim, m³/s)', axis=1, inplace=False)

In [ ]:
# output a pandas dataframe with all metric values for a given list of metrics and stations
sim=routing_irr
metric_list=[metric_sim_module, metric_obs_module, metric_bias, metric_rmse, metric_tcorr, metric_nse, metric_kge]
stations_dict=proper_stations_dict
# define dataframe with one row per station and one column per metric
df=[]
for station_id, station in stations_dict.items():
    label='{} ({})'.format(station['station_nb'],station['name'])
    df.append({'Station':label})
    for metric in metric_list:
        name=metric.__short_name__
        metric_value=compute_metric_station(sim, obs_routing, station_id, station, metric)
        metric_value=np.round(metric_value, 2)
        # append metric_value to df
        df[-1][name]=metric_value

# convert df to pandas dataframe
df_irr=pd.DataFrame(df)
# set Station as index
df_irr.set_index('Station', inplace=True)
df_irr['Bias (%)'] = np.round(df_irr['Bias (m³/s)'] / df_irr['Module (obs, m³/s)'], 3) * 100
#move Bias (%) to the 4th column
cols = df_irr.columns.tolist()
cols = cols[:3] + cols[-1:] + cols[3:-1]
df_irr = df_irr[cols]
#add average row
df_irr.loc['Mean'] = df_irr.mean()
df_irr.drop('Module (obs, m³/s)', axis=1, inplace=False).drop('Module (sim, m³/s)', axis=1, inplace=False)

In [ ]:
df_diff=df_irr-df_noirr
df_diff['Bias change (%)'] = np.round(df_diff['Bias (m³/s)'] / df_noirr['Bias (m³/s)'], 3) * 100
df_diff['RMSE change (%)'] = np.round(df_diff['RMSE'] / df_noirr['RMSE'], 4) * 100
df_diff.drop('Module (obs, m³/s)', axis=1, inplace=True)
df_diff.drop('Module (sim, m³/s)', axis=1, inplace=True)
df_diff.drop('Bias (m³/s)', axis=1, inplace=True)
df_diff.drop('Bias (%)', axis=1, inplace=True)
df_diff.drop('RMSE', axis=1, inplace=True)
df_diff

In [ ]:
#export to csv to put in latex
df_noirr.to_csv('figures/df_noirr.csv')
df_irr.to_csv('figures/df_irr.csv')
df_diff.to_csv('figures/df_diff.csv')

## Tab 4 : Subdomains of different irrigation intensity


Areas (computed here) and mean irrigation (see Fig.9)

Thresholds defined in Fig.9 as well, 0.06 mm/d and 0.12 mm/d

```
irr_mask_low    = ip_ORCirr.mean(dim='time')['irrigation']<0.06
irr_mask_high   = ip_ORCirr.mean(dim='time')['irrigation']>0.12
irr_mask_med    = (~irr_mask_low) * (~irr_mask_high) * ip_mask
```

In [ ]:
ds_area= compute_grid_cell_width(ip_ORCirr)

mask1=irr_mask_low
mask_area_low=compute_mask_area(ds_area, mask1)

mask2=irr_mask_med
mask_area_med=compute_mask_area(ds_area, mask2)

mask3=irr_mask_high
mask_area_high=compute_mask_area(ds_area, mask3)

mask4=ip_mask
mask_area_ip=compute_mask_area(ds_area, mask4)

In [ ]:
#fraction of IP for each area
#low irr
lowirr_fraction=mask_area_low / mask_area_ip
#medium irr
medirr_fraction=mask_area_med / mask_area_ip
#high irr
highirr_fraction=mask_area_high / mask_area_ip
#total fraction of IP
ip_fraction=mask_area_ip / mask_area_ip
#print all
print('Low irrigation fraction: {}'.format(lowirr_fraction))
print('Medium irrigation fraction: {}'.format(medirr_fraction))
print('High irrigation fraction: {}'.format(highirr_fraction))
print('Iberian Peninsula fraction: {}'.format(ip_fraction))

# Appendix

In [ ]:
plot_appendix=True

## Offline calibration of irrigation SM target

In [ ]:
stations_calib_v0 = {
    6298249: {'name': 'Zaragoza',               'river': 'Ebro',        'lon_grid':-0.8749926686286926, 'lat_grid':41.65833282470703,   'year_min':2003, 'year_max':2012, 'station_nb':1},
    6298992: {'name': 'Albarreal De Tajo',      'river': 'Tagus',       'lon_grid':-4.17499303817749,   'lat_grid':39.891666412353516,  'year_min':2003, 'year_max':2012, 'station_nb':2},
    6298481: {'name': 'San Miguel del Pino',    'river': 'Douro',       'lon_grid':-4.92499303817749,   'lat_grid':41.508331298828125,  'year_min':2003, 'year_max':2012, 'station_nb':3},
    6298259: {'name': 'Badajoz',                'river': 'Guadiana',    'lon_grid': -7.008326530456543, 'lat_grid': 38.85833358764648,  'year_min':2003, 'year_max':2012, 'station_nb':4},
                }

In [ ]:
filename = '../../../obs/streamflow/GRDC_Monthly_Jan20_v1_ES.nc'
obs_routing_polcher = xr.open_dataset(filename)
rename_dict = {'hydrographs': 'runoff_mean',
               'number': 'id',}
obs_routing_polcher = obs_routing_polcher.rename(rename_dict)
#shift time coordinate to match the sims and grdc obs
obs_routing_polcher['time'] = obs_routing_polcher['time'] - np.timedelta64(14, 'D')
# obs_routing_polcher

In [ ]:
dir='../../../JZ_simu_outputs/routing_native'
spinupyears_long = [1980, 1981, 1982]

In [ ]:
def format_routing_ORC(filename, name, spinupyears, color=None):
    ds = xr.open_mfdataset(filename)
    ds = ds.rename({'time_counter': 'time'})
    ds.attrs['name'] = name
    ds = remove_years_efficiently(ds, spinupyears)

    #edit units and names
    if 'hydrographs' in ds:
        ds['hydrographs'].attrs['units'] = 'm³ s⁻¹'
        ds['hydrographs'].attrs['long_name'] = 'River discharge'
    if 'streamr' in ds:
        ds['streamr'].attrs['units'] = 'kg m⁻²'
        ds['streamr'].attrs['long_name'] = 'River reservoir volume'
    if 'slowr' in ds:
        ds['slowr'].attrs['units'] = 'kg m⁻²'
        ds['slowr'].attrs['long_name'] = 'GW reservoir volume'
    if 'fastr' in ds:
        ds['fastr'].attrs['units'] = 'kg m⁻²'
        ds['fastr'].attrs['long_name'] = 'Overland reserv. volume'

    if color is not None:
        ds.attrs['plot_color'] = color

    return ds
def format_routing_nativeDIAG(filename, name, spinupyears, color=None):
    ds = xr.open_mfdataset(filename)
    ds.attrs['name'] = name

    dict = {'time_counter' : 'time',
            
            # for reservoirs
            'lat_domain_landpoints_routing':'lat',
            'lon_domain_landpoints_routing':'lon',
            # for drainage and runoff
            # 'lat_domain_landpoints':'lat',
            # 'lon_domain_landpoints':'lon',

            'routing_stream_reservoir' : 'streamr',
            'routing_fast_reservoir' : 'fastr',
            'routing_slow_reservoir' : 'slowr',
            'routing_drainage' : 'drainage',
            'routing_runoff' : 'runoff',
            'routing_riverflow' : 'riverflow',
            'routing_coastalflow' : 'coastalflow',
            'routing_irrigation' : 'irrigation',}
    #change the variables that exist according to the dictionary
    ds = ds.rename(dict)
    #remove first years to consider a spinup
    ds = remove_years_efficiently(ds, spinupyears)

    if color is not None:
        ds.attrs['plot_color'] = color

    return(ds)
def format_routing_native(filename, name, spinupyears, color=None):
    ds = xr.open_mfdataset(filename)
    ds.attrs['name'] = name

    dict = {'time_counter' : 'time',
        'routing_stream_reservoir_r' : 'streamr',
        'routing_fast_reservoir_r' : 'fastr',
        'routing_slow_reservoir_r' : 'slowr',
        'routing_drainage_r' : 'drainage',
        'routing_runoff_r' : 'runoff',
        'routing_hydrographs_r' : 'hydrographs',
        'routing_riverflow_r' : 'riverflow',
        'routing_coastalflow_r' : 'coastalflow'
        }
    ds = ds.rename(dict)
    ds = ds.isel(lon=slice(2,-2),lat=slice(2,-2))

    ds = remove_years_efficiently(ds, spinupyears)

    if color is not None:
        ds.attrs['plot_color'] = color

    #edit units and names
    if 'hydrographs' in ds:
        ds['hydrographs'].attrs['units'] = 'm³ s⁻¹'
        ds['hydrographs'].attrs['long_name'] = 'River discharge'
    if 'streamr' in ds:
        ds['streamr'].attrs['units'] = 'kg m⁻²'
        ds['streamr'].attrs['long_name'] = 'River reservoir volume'
    if 'slowr' in ds:
        ds['slowr'].attrs['units'] = 'kg m⁻²'
        ds['slowr'].attrs['long_name'] = 'GW reservoir volume'
    if 'fastr' in ds:
        ds['fastr'].attrs['units'] = 'kg m⁻²'
        ds['fastr'].attrs['long_name'] = 'Overland reserv. volume'
        
    #edit units and names
    ds['hydrographs'].attrs['units'] = 'm³ s⁻¹'
    ds['hydrographs'].attrs['long_name'] = 'River discharge'

    #to move all to 1st day of the month (instead of 14, 15, or 16..)
    ds = ds.resample(time='MS').mean()
    return ds

In [ ]:
#noirr
filename = '{}/calibration_MERIT/tcst7_long/tcst7*_1D_diag_routing_r.nc'.format(dir)
name = 'no_irr'
DIAGR_tcst4_long = format_routing_native(filename, name, spinupyears_long, color='red')

#irr default
filename = '{}/calibration_MERIT/tcst7irr_long/nat18*_1D_diag_routing_r.nc'.format(dir)
name = 'irr_default_beta0.9'
DIAGR_tcst4_irr_long = format_routing_native(filename, name, spinupyears_long, color='blue')

#irr_reduced
filename = '{}/calibration_MERIT/tcst7irr0.6_long/tcst7irr0.6*_1D_diag_routing_r.nc'.format(dir)
name = 'irr_reduced_beta0.6'
DIAGR_tcst4_irr_reduced_long = format_routing_native(filename, name, spinupyears_long, color='deepskyblue')

In [ ]:
stations_dict = stations_calib_v0
obs_ds= obs_routing_polcher
polcher_ds = True
fig, axes= plt.subplots(2,2, figsize=(16,8))
axes= axes.flatten()
plot_all_sim=False

ds1 = DIAGR_tcst4_long
ds2 = DIAGR_tcst4_irr_long
ds3 = DIAGR_tcst4_irr_reduced_long

ds_list = [ds1, ds2, ds3]

for i, (station_id, station_data) in enumerate(stations_dict.items()):
    title_letter=letters[i]
    sc_station(obs_ds,axes[i], station_id, name=station_data['name'], ylabel='River discharge (m³ s⁻¹)', year_min=1983, year_max=2010, polcher_ds=polcher_ds)
    sc_with_obs(ds_list, obs_ds, axes[i], station_id, station_data, ylabel='River discharge (m³ s⁻¹)', year_min=1983, year_max=2010, polcher_ds=polcher_ds, plot_all_sim=plot_all_sim, title_letter=title_letter, title_number=False)

## 18 stations time series and seasonal cycle

In [ ]:
if plot_appendix:
    #plot a time series for each station from dict
    fig, axes= plt.subplots(6,3, figsize=(20,28))
    axes= axes.flatten()
    ds_list=[routing_noirr, routing_irr]
    for i, (station_id, station_data) in enumerate(proper_stations_dict.items()):
                ts_station(obs_routing,axes[i], station_id, name=station_data['name'], year_min=2010, year_max=2017)
                ts_with_obs(ds_list, obs_routing, axes[i], station_id, station_data, year_min=2010, year_max=2017, plot_all_sim=True)

In [ ]:
if plot_appendix:
    #plot a time series for each station from dict
    fig, axes= plt.subplots(6,3, figsize=(20,28))
    axes= axes.flatten()
    ds_list=[routing_noirr, routing_irr]
    for i, (station_id, station_data) in enumerate(proper_stations_dict.items()):
        sc_station(obs_routing,axes[i], station_id, name=station_data['name'], year_min=2010, year_max=2017)
        sc_with_obs(ds_list, obs_routing, axes[i], station_id, station_data, year_min=2010, year_max=2017)

## Impacts of Fig. 7 in other seasons

In [ ]:
#MAM
vars=['irrigation', 'evap_frac', 't2m', 's_pblh', 's_lcl', 'precip', 'cape', 'moisture_convergence']
# vmins=[0  , -0.3, -0.35, -100, -250, -1, -35, -1]
# vmaxs=[1.5,  0.3,  0.35,  100,  250,  1,  35,  1]
# vmins=[0  , -0.1, -0.1, -30, -70, -0.2, -3.5, -0.3]
# vmaxs=[0.5,  0.1,  0.1,  30,  70,  0.2,  3.5,  0.3]
vmaxs=[0.5,  0.2,  0.2,  40,  100,  0.25,  5,  0.5]
vmins=[0  , -0.2, -0.2, -40, -100, -0.25, -5, -0.5]
clabels=['Irrigation (mm d⁻¹)',
         'Evaporative fraction change',
         '2-m temperature change (K)',
         'ABL height change (m)',
         'LCL change (m)',
         'Precipitation change (mm d⁻¹)',
         'CAPE change (J kg⁻¹)', 
         'Convergence change (mm d⁻¹)']
cmaps=[wet, emb_neutral, emb, emb, emb, emb_neutral, emb, emb_neutral]

fig, axes = plt.subplots(4, 2, figsize=(17,16), subplot_kw={'projection': ccrs.PlateCarree()})
axes = axes.flatten()

months=[3,4,5]

i=0
map_wind(ip_simirr.sel(time=ip_simirr['time.month'].isin(months)), 
         axes[i], 
         extra_var=vars[i], 
         extra_ds=ip_ORCirr.sel(time=ip_ORCirr['time.month'].isin(months)).mean(dim='time'),
         height='10m', dist=2, scale=30,
         cmap=cmaps[i], 
         vmin=vmins[i],vmax=vmaxs[i],
           clabel=clabels[i],
         quiver_inside=True)

pvalue=0.05
sig_method=1
check_norm=False
sig_viz=6
hatch='///'

for i in range(1,2):
    ds1=ip_ORCirr.sel(time=ip_ORCirr['time.month'].isin(months))
    ds2=ip_ORCnoirr.sel(time=ip_ORCnoirr['time.month'].isin(months))
    var=vars[i]
    diff = (ds1[var] - ds2[var]).mean(dim='time')
    plotvar= diff
    ax=axes[i]
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel=clabels[i]
    cmap=cmaps[i]
    sig_mask = compute_sig_mask(ds1, ds2, var, check_norm=check_norm, method=sig_method, pvalue=pvalue)
    nice_map(plotvar, ax, cmap=cmap, 
             vmin=vmin, vmax=vmax, 
             sig_mask=sig_mask, 
             clabel=clabel, hatch=hatch)


for i in range(2, 8):
    ds1=ip_simirr.sel(time=ip_simirr['time.month'].isin(months))
    ds2=ip_sim.sel(time=ip_sim['time.month'].isin(months))
    var=vars[i]
    diff = (ds1[var] - ds2[var]).mean(dim='time')
    plotvar= diff
    ax=axes[i]
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel=clabels[i]
    cmap=cmaps[i]
    sig_mask = compute_sig_mask(ds1, ds2, var, check_norm=check_norm, method=sig_method, pvalue=pvalue)
    nice_map(plotvar, ax, cmap=cmap, 
             vmin=vmin, vmax=vmax, 
             sig_mask=sig_mask,
             clabel=clabel, hatch=hatch)


for i in range(8):
    title='({})'.format(letters[i])
    axes[i].set_title(title)

if True:
    # plt.savefig('figures/article/appendix_f07_MAM_cbarJJA.png', bbox_inches='tight', dpi=300)
    plt.savefig('figures/article/appendix_f07_MAM_cbar_common.png', bbox_inches='tight', dpi=300)
    # plt.savefig('figures/article/appendix_f07_MAM_nocbarlimit.png', bbox_inches='tight', dpi=300)
    # plt.savefig('figures/article/appendix_f07_MAM.png', bbox_inches='tight', dpi=300)

In [ ]:
# SON
vars=['irrigation', 'evap_frac', 't2m', 's_pblh', 's_lcl', 'precip', 'cape', 'moisture_convergence']
# vmins=[0  , -0.3, -0.35, -100, -250, -1, -35, -1]
# vmaxs=[1.5,  0.3,  0.35,  100,  250,  1,  35,  1]
# vmins=[0  , -0.2, -0.2, -40, -120, -0.25, -5, -0.4]
# vmaxs=[0.5,  0.2,  0.2,  40,  120,  0.25,  5,  0.4]
vmaxs=[0.5,  0.2,  0.2,  40,  100,  0.25,  5,  0.5]
vmins=[0  , -0.2, -0.2, -40, -100, -0.25, -5, -0.5]
clabels=['Irrigation (mm d⁻¹)',
         'Evaporative fraction change',
         '2-m temperature change (K)',
         'ABL height change (m)',
         'LCL change (m)',
         'Precipitation change (mm d⁻¹)',
         'CAPE change (J kg⁻¹)', 
         'Convergence change (mm d⁻¹)']
cmaps=[wet, emb_neutral, emb, emb, emb, emb_neutral, emb, emb_neutral]

fig, axes = plt.subplots(4, 2, figsize=(17,16), subplot_kw={'projection': ccrs.PlateCarree()})
axes = axes.flatten()

months=[9,10,11]

i=0
map_wind(ip_simirr.sel(time=ip_simirr['time.month'].isin(months)), 
         axes[i], 
         extra_var=vars[i], 
         extra_ds=ip_ORCirr.sel(time=ip_ORCirr['time.month'].isin(months)).mean(dim='time'),
         height='10m', dist=2, scale=30,
         cmap=cmaps[i], 
         vmin=vmins[i],vmax=vmaxs[i],
           clabel=clabels[i],
         quiver_inside=True)

pvalue=0.05
sig_method=1
check_norm=False
sig_viz=6
hatch='///'

for i in range(1,2):
    ds1=ip_ORCirr.sel(time=ip_ORCirr['time.month'].isin(months))
    ds2=ip_ORCnoirr.sel(time=ip_ORCnoirr['time.month'].isin(months))
    var=vars[i]
    diff = (ds1[var] - ds2[var]).mean(dim='time')
    plotvar= diff
    ax=axes[i]
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel=clabels[i]
    cmap=cmaps[i]
    sig_mask = compute_sig_mask(ds1, ds2, var, check_norm=check_norm, method=sig_method, pvalue=pvalue)
    nice_map(plotvar, ax, cmap=cmap, 
             vmin=vmin, vmax=vmax, 
             sig_mask=sig_mask, 
             clabel=clabel, hatch=hatch)


for i in range(2, 8):
    ds1=ip_simirr.sel(time=ip_simirr['time.month'].isin(months))
    ds2=ip_sim.sel(time=ip_sim['time.month'].isin(months))
    var=vars[i]
    diff = (ds1[var] - ds2[var]).mean(dim='time')
    plotvar= diff
    ax=axes[i]
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel=clabels[i]
    cmap=cmaps[i]
    sig_mask = compute_sig_mask(ds1, ds2, var, check_norm=check_norm, method=sig_method, pvalue=pvalue)
    nice_map(plotvar, ax, cmap=cmap, 
             vmin=vmin, vmax=vmax, 
             sig_mask=sig_mask, 
             clabel=clabel, hatch=hatch)


for i in range(8):
    title='({})'.format(letters[i])
    axes[i].set_title(title)

if True:
    # plt.savefig('figures/article/appendix_f07_SON_cbarJJA.png', bbox_inches='tight', dpi=300)
    plt.savefig('figures/article/appendix_f07_SON_cbar_common.png', bbox_inches='tight', dpi=300)
    # plt.savefig('figures/article/appendix_f07_SON_nocbarlimit.png', bbox_inches='tight', dpi=300)
    # plt.savefig('figures/article/appendix_f07_SON.png', bbox_inches='tight', dpi=300)

In [ ]:
#DJF
vars=['irrigation', 'evap_frac', 't2m', 's_pblh', 's_lcl', 'precip', 'cape', 'moisture_convergence']
# vmins=[0  , -0.3, -0.35, -100, -250, -1, -35, -1]
# vmaxs=[1.5,  0.3,  0.35,  100,  250,  1,  35,  1]
# vmins=[0   , -0.1, -0.04, -5, -15, -0.01, -0.2, -0.08]
# vmaxs=[0.03,  0.1,  0.04,  5,  15,  0.01,  0.2,  0.08]
vmaxs=[0.5,  0.2,  0.2,  40,  100,  0.25,  5,  0.5]
vmins=[0  , -0.2, -0.2, -40, -100, -0.25, -5, -0.5]
clabels=['Irrigation (mm d⁻¹)',
         'Evaporative fraction change',
         '2-m temperature change (K)',
         'ABL height change (m)',
         'LCL change (m)',
         'Precipitation change (mm d⁻¹)',
         'CAPE change (J kg⁻¹)', 
         'Convergence change (mm d⁻¹)']
cmaps=[wet, emb_neutral, emb, emb, emb, emb_neutral, emb, emb_neutral]

fig, axes = plt.subplots(4, 2, figsize=(17,16), subplot_kw={'projection': ccrs.PlateCarree()})
axes = axes.flatten()

months=[1,2,12]

i=0
map_wind(ip_simirr.sel(time=ip_simirr['time.month'].isin(months)), 
         axes[i], 
         extra_var=vars[i], 
         extra_ds=ip_ORCirr.sel(time=ip_ORCirr['time.month'].isin(months)).mean(dim='time'),
         height='10m', dist=2, scale=30,
         cmap=cmaps[i], 
         vmin=vmins[i],vmax=vmaxs[i],
           clabel=clabels[i],
         quiver_inside=True)

pvalue=0.05
sig_method=1
check_norm=False
sig_viz=6
hatch='///'

for i in range(1,2):
    ds1=ip_ORCirr.sel(time=ip_ORCirr['time.month'].isin(months))
    ds2=ip_ORCnoirr.sel(time=ip_ORCnoirr['time.month'].isin(months))
    var=vars[i]
    diff = (ds1[var] - ds2[var]).mean(dim='time')
    plotvar= diff
    ax=axes[i]
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel=clabels[i]
    cmap=cmaps[i]
    sig_mask = compute_sig_mask(ds1, ds2, var, check_norm=check_norm, method=sig_method, pvalue=pvalue)
    nice_map(plotvar, ax, cmap=cmap, 
             vmin=vmin, vmax=vmax, 
             sig_mask=sig_mask, 
             clabel=clabel, hatch=hatch)

for i in range(2, 8):
    ds1=ip_simirr.sel(time=ip_simirr['time.month'].isin(months))
    ds2=ip_sim.sel(time=ip_sim['time.month'].isin(months))
    var=vars[i]
    diff = (ds1[var] - ds2[var]).mean(dim='time')
    plotvar= diff
    ax=axes[i]
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel=clabels[i]
    cmap=cmaps[i]
    sig_mask = compute_sig_mask(ds1, ds2, var, check_norm=check_norm, method=sig_method, pvalue=pvalue)
    nice_map(plotvar, ax, cmap=cmap, 
             vmin=vmin, vmax=vmax, 
             sig_mask=sig_mask,
            clabel=clabel, hatch=hatch)

for i in range(8):
    title='({})'.format(letters[i])
    axes[i].set_title(title)

if True:
    # plt.savefig('figures/article/appendix_f07_DJF_cbarJJA.png', bbox_inches='tight', dpi=300)
    plt.savefig('figures/article/appendix_f07_DJF_cbar_common.png', bbox_inches='tight', dpi=300)
    # plt.savefig('figures/article/appendix_f07_DJF.png', bbox_inches='tight', dpi=300)
    # plt.savefig('figures/article/appendix_f07_DJF_nocbarlimit.png', bbox_inches='tight', dpi=300)

## Snowmelt

In [ ]:
fig = plt.figure(figsize=(16, 4))
gs = gridspec.GridSpec(1, 2, width_ratios=[1,1.2])


ax1 = fig.add_subplot(gs[0,0])
ax2 = fig.add_subplot(gs[0,1], projection=ccrs.PlateCarree())

#Mean seasonal cycle (2010-2022)

ds1 = ip_ORCnoirr
plotvar1 = ds1['rain'].mean(dim=['lon','lat'])
plotvar2 = ds1['snowf'].mean(dim=['lon','lat'])
plotvar3 = ds1['snowmelt'].mean(dim=['lon','lat'])

ds2 = ip_sim
plotvar4 = ds2['precip'].mean(dim=['lon','lat'])

plotvars= [plotvar1, plotvar2, plotvar3, plotvar4]
labels=['Rainfall', 'Snowfall', 'Snowmelt', 'Total precipitation']

seasonal_cycle(plotvars, labels, ax=ax1, ylabel = 'mm d⁻¹')

# Map of snowfall / (snowfall + rain)

plotvar=ds1['snow_contrib'].mean(dim='time')
cmap=bluesW
vmin=0
vmax=35
clabel="Share of snowfall in precipitation (%)"
nice_map(plotvar, ax2, cmap, vmin, vmax, clabel=clabel)


ax1.set_title('(a)')
ax2.set_title('(b)')

## Spinup

### Load files

In [ ]:
#add spinup
filename = '{}/*/SRF/*history.nc'.format(noirr_dir)
spinup_noirr = format_ORC_MO(filename, 'spinup_no_irr', 'grey',
                         lon_min, lon_max, lat_min, lat_max)
spinup_noirr

In [ ]:
filename = '{}/*/SRF/*history.nc'.format(irr_dir)
spinup_irr = format_ORC_MO(filename, 'spinup_irr', 'grey',
                         lon_min, lon_max, lat_min, lat_max)
spinup_irr

In [ ]:
ip_spinup_noirr = spinup_noirr.where(con_mask).where(ip_mask)
ip_spinup_noirr['time'] = ip_spinup_noirr['time'] - pd.to_timedelta(5 * 365.25, unit='D')

ip_spinup_irr = spinup_irr.where(con_mask).where(ip_mask)
ip_spinup_irr['time'] = ip_spinup_irr['time'] - pd.to_timedelta(5 * 365.25, unit='D')

### Figures

In [ ]:
vars=['precip', 'tot_runoff', 'evap', 'humtot', 'slowr', 'streamr', 'irrigation', 'LAImean']
clabels=['Precipiation (mm d⁻¹)',
         'Total runoff (mm d⁻¹)',
         'ET (mm d⁻¹)',
         'Soil moisture (mm)',
         'Groundwater reservoir (mm)',
         'River reservoir (mm)',
         'Irrigation (mm d⁻¹)',
         'Mean LAI (-)']
legends=[False, None, None, None, None, None, None, None]

ds1=ip_spinup_irr
ds2=ip_ORCirr
ds3=ip_spinup_noirr
ds4=ip_ORCnoirr

ds_list=[ds1, ds2]
# ds_list=[ds1, ds2, ds3, ds4]

fig, axes = plt.subplots(4, 2, figsize=(16,16))
axes = axes.flatten()

for i in range(8):
    var=vars[i]
    ax=axes[i]
    clabel=clabels[i]
    legend=legends[i]
    title='({})'.format(letters[i])
    annual_bars_ave(ds_list, var, ax=ax, ds_colors=True,
                    year_min=2007, year_max=2022, 
                    ylabel=clabel, legend_out=legend, alpha=1.0)
    ax.set_title('')
    ax.text(
        0., 
        1.05, 
        title, 
        transform=ax.transAxes, # Essential: Use relative coordinates (0 to 1)
        fontsize=16, 
        # fontweight='bold',
        va='bottom',
        ha='left'
)

### Table of correlation coefficients

In [ ]:
def calculate_pvalues_matrix(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the pairwise two-sided p-value matrix for Pearson correlation 
    coefficients in a pandas DataFrame.
    """
    
    df_pvalues = pd.DataFrame(index=df.columns, columns=df.columns, dtype=float)
    cols = df.columns
    
    for i in range(len(cols)):
        for j in range(i, len(cols)):
            col1 = cols[i]
            col2 = cols[j]
            
            # pearsonr returns (correlation_coefficient, p-value)
            _, p_value = pearsonr(df[col1], df[col2])
            
            # Fill the p-value into the matrix symmetrically
            df_pvalues.loc[col1, col2] = p_value
            df_pvalues.loc[col2, col1] = p_value
            
    return df_pvalues.round(4)

In [ ]:
ds = ip_ORCirr
all_vars = ['precip', 'tot_runoff', 'evap', 'humtot', 'slowr', 'streamr', 'irrigation', 'LAImean']

# Descriptive labels for the resulting DataFrame columns and index
new_labels = [
    'Precipiation',
    'Total runoff',
    'ET',
    'Soil moisture',
    'Groundwater reservoir',
    'River reservoir',
    'Irrigation',
    'Mean LAI'
]
name_mapping = dict(zip(all_vars, new_labels))

# 1. Process all variables into a dictionary of annual time series
annual_series = {
    var: ds[var].mean(dim=['lon','lat']).resample(time='1YE').mean() 
    for var in all_vars if var in ds
}

# 2. Convert xarray DataArrays to a Pandas DataFrame
df_annual = pd.DataFrame({
    name: data.to_series() for name, data in annual_series.items()
})

# 3. Rename columns using the descriptive labels
df_annual = df_annual.rename(columns=name_mapping)

# 4. Calculate the full pairwise correlation matrix
corr_matrix = df_annual.corr().round(3)

# 5. Mask the lower triangle and the diagonal (k=1 starts above the main diagonal)
mask = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
corr_matrix_upper = corr_matrix.where(mask)


# 6. Calculate the full pairwise p-value matrix
p_value_matrix = calculate_pvalues_matrix(df_annual)

# 7. Mask the lower triangle and the diagonal of the p-value matrix
p_value_matrix_upper = p_value_matrix.where(mask)


# 8. Create a new matrix with significance stars appended to the correlation coefficient
corr_matrix_starred_upper = corr_matrix_upper.copy().astype(str)

for i in corr_matrix_upper.index:
    for j in corr_matrix_upper.columns:
        r = corr_matrix_upper.loc[i, j]
        p = p_value_matrix_upper.loc[i, j]
        
        # Only process if r is a valid number (i.e., not NaN from masking)
        if pd.notna(r):
            stars = ''
            if p < 0.001:
                stars = '***'
            elif p < 0.01:
                stars = '**'
            elif p < 0.05:
                stars = '*'
            
            # Format the correlation coefficient and append stars
            # Use f-string formatting to control the decimal places
            corr_matrix_starred_upper.loc[i, j] = f"{r:.2f}{stars}"
        else:
             # Preserve NaN values from the mask
            corr_matrix_starred_upper.loc[i, j] = np.nan

# corr_matrix_upper 
corr_matrix_starred_upper

In [ ]:
ds = ip_ORCnoirr
all_vars = ['precip', 'tot_runoff', 'evap', 'humtot', 'slowr', 'streamr', 'LAImean']

# Descriptive labels for the resulting DataFrame columns and index
new_labels = [
    'Precipiation',
    'Total runoff',
    'ET',
    'Soil moisture',
    'Groundwater reservoir',
    'River reservoir',
    'Mean LAI'
]
name_mapping = dict(zip(all_vars, new_labels))

# 1. Process all variables into a dictionary of annual time series
annual_series = {
    var: ds[var].mean(dim=['lon','lat']).resample(time='1YE').mean() 
    for var in all_vars if var in ds
}

# 2. Convert xarray DataArrays to a Pandas DataFrame
df_annual = pd.DataFrame({
    name: data.to_series() for name, data in annual_series.items()
})

# 3. Rename columns using the descriptive labels
df_annual = df_annual.rename(columns=name_mapping)

# 4. Calculate the full pairwise correlation matrix
corr_matrix = df_annual.corr().round(3)

# 5. Mask the lower triangle and the diagonal (k=1 starts above the main diagonal)
mask = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
corr_matrix_upper = corr_matrix.where(mask)

# 6. Calculate the full pairwise p-value matrix
p_value_matrix = calculate_pvalues_matrix(df_annual)

# 7. Mask the lower triangle and the diagonal of the p-value matrix
p_value_matrix_upper = p_value_matrix.where(mask)


# 8. Create a new matrix with significance stars appended to the correlation coefficient
corr_matrix_starred_upper = corr_matrix_upper.copy().astype(str)

for i in corr_matrix_upper.index:
    for j in corr_matrix_upper.columns:
        r = corr_matrix_upper.loc[i, j]
        p = p_value_matrix_upper.loc[i, j]
        
        # Only process if r is a valid number (i.e., not NaN from masking)
        if pd.notna(r):
            stars = ''
            if p < 0.001:
                stars = '***'
            elif p < 0.01:
                stars = '**'
            elif p < 0.05:
                stars = '*'
            
            # Format the correlation coefficient and append stars
            # Use f-string formatting to control the decimal places
            corr_matrix_starred_upper.loc[i, j] = f"{r:.2f}{stars}"
        else:
             # Preserve NaN values from the mask
            corr_matrix_starred_upper.loc[i, j] = np.nan

# corr_matrix_upper 
corr_matrix_starred_upper

## Difference to products by season 

In [ ]:
#map for 4 seasons
var='evap'

date_min = '2010-01-01'
date_max = '2019-12-31'

ds1=ip_sim.sel(time=slice(date_min,date_max))
ds2=ip_gleam_isim.sel(time=slice(date_min,date_max))

max_value= 1.5
min_value=-1.5

cmap=emb_neutral

diff=ds1[var]-ds2[var]
plotvar=diff

clabel='ET bias (mm d⁻¹)'

map_seasons(plotvar, cmap=cmap, figsize=(16, 8), vmin=min_value, vmax=max_value, clabel=clabel)

In [ ]:
#map for 4 seasons
var='precip'

date_min = '2010-01-01'
date_max = '2019-12-31'

ds1=ip_sim.sel(time=slice(date_min,date_max))
ds2=ip_gpcc_isim.sel(time=slice(date_min,date_max))

max_value= 1.5
min_value=-1.5

cmap=emb_neutral

diff=ds1[var]-ds2[var]
plotvar=diff

clabel='Precipitation bias (mm d⁻¹)'

map_seasons(plotvar, cmap=cmap, figsize=(16, 8), vmin=min_value, vmax=max_value, clabel=clabel)

In [ ]:
#map for 4 seasons
var='norm_sm'

date_min = '2010-01-01'
date_max = '2019-12-31'

ds1=normalize_sm(ip_ORCnoirr, 'mrsos', date_min, date_max)
ds2=normalize_sm(ip_cci_iORC, 'sm', date_min, date_max)

max_value= 1.5
min_value=-1.5

cmap=emb_neutral

diff=ds1[var]-ds2[var]
plotvar=diff

clabel='Normalized SSM bias (-)'

map_seasons(plotvar, cmap=cmap, figsize=(16, 8), vmin=min_value, vmax=max_value, clabel=clabel)

## Absolute value of SSM in irr and no_irr

In [ ]:
fig = plt.figure(figsize=(16, 4))
gs = gridspec.GridSpec(1, 2, width_ratios=[1,1.2])


ax1 = fig.add_subplot(gs[0,0])
ax2 = fig.add_subplot(gs[0,1], projection=ccrs.PlateCarree())

#Mean seasonal cycle (2010-2022)
var='mrsos'
ds1 = ip_ORCnoirr
ds2= ip_ORCirr
labels=['no_irr', 'irr']
colors=['red', 'blue']
plotvars=[ds1[var].mean(dim=['lon','lat']), 
          ds2[var].mean(dim=['lon','lat'])]
ylabel = 'Surface soil moisture (mm)'

# seasonal_cycle(plotvars, var, ax=ax1, ylabel=ylabel)
seasonal_cycle(plotvars, labels, ax=ax1, colors=colors, ylabel=ylabel)

# Map of diff / (snowfall + rain)

diff = ((ds2[var] - ds1[var]) / ds1[var] * 100).mean(dim='time')
cmap=emb_neutral
vmin=-10
vmax= 10
clabel="SSM relative difference (%)"
nice_map(diff, ax2, cmap, vmin, vmax, clabel=clabel)

ax1.set_title('(a)')
ax2.set_title('(b)')